In [1]:
import pandas as pd

df = pd.read_excel('stat_apiculture.xls', converters={'dep':str})
df

,reg,dep,tranche,nb_decl,nb_decl_siret,nb_decl_sans_siret,nb_ruches
0,AUVERGNE-RHONE-ALPES,01,1 à 10 ruches,548,144,404,2998
1,AUVERGNE-RHONE-ALPES,01,11 à 49 ruches,184,109,75,3692
2,AUVERGNE-RHONE-ALPES,01,50 à 199 ruches,31,30,1,2906
3,AUVERGNE-RHONE-ALPES,01,200 à 499 ruches,16,16,0,4417
4,AUVERGNE-RHONE-ALPES,01,500 à 999 ruches,9,9,0,5557
...,...,...,...,...,...,...,...
502,PROVENCE-ALPES-COTE D'AZUR,84,1 à 10 ruches,243,41,202,1351
503,PROVENCE-ALPES-COTE D'AZUR,84,11 à 49 ruches,123,55,68,2776
504,PROVENCE-ALPES-COTE D'AZUR,84,50 à 199 ruches,45,36,9,4559
505,PROVENCE-ALPES-COTE D'AZUR,84,200 à 499 ruches,51,49,2,14239


In [2]:
import unidecode

res = df.pivot(index=['reg', 'dep'], columns='tranche', values=['nb_decl', 'nb_decl_siret', 'nb_decl_sans_siret', 'nb_ruches'])
res = res.reset_index()
print(res.columns)
res.columns = [unidecode.unidecode('_'.join(col).rstrip('_').replace(' ', '_')) for col in res.columns.values]
res['tot_ruches'] = res['nb_ruches_1_a_10_ruches'].fillna(0) + res['nb_ruches_11_a_49_ruches'].fillna(0) + \
             res['nb_ruches_200_a_499_ruches'].fillna(0) + res['nb_ruches_50_a_199_ruches'].fillna(0) + \
             res['nb_ruches_500_a_999_ruches'].fillna(0) + res['nb_ruches_plus_de_1000_ruches'].fillna(0)
# res.to_excel('ruches.xls')

MultiIndex([(               'reg',                    ''),
            (               'dep',                    ''),
            (           'nb_decl',       '1 à 10 ruches'),
            (           'nb_decl',      '11 à 49 ruches'),
            (           'nb_decl',    '200 à 499 ruches'),
            (           'nb_decl',     '50 à 199 ruches'),
            (           'nb_decl',    '500 à 999 ruches'),
            (           'nb_decl', 'plus de 1000 ruches'),
            (     'nb_decl_siret',       '1 à 10 ruches'),
            (     'nb_decl_siret',      '11 à 49 ruches'),
            (     'nb_decl_siret',    '200 à 499 ruches'),
            (     'nb_decl_siret',     '50 à 199 ruches'),
            (     'nb_decl_siret',    '500 à 999 ruches'),
            (     'nb_decl_siret', 'plus de 1000 ruches'),
            ('nb_decl_sans_siret',       '1 à 10 ruches'),
            ('nb_decl_sans_siret',      '11 à 49 ruches'),
            ('nb_decl_sans_siret',    '200 à 499 ruches'

/tmp/ipykernel_16168/164023161.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  res.to_excel('out.xls')


In [4]:
import numpy as np
import geopandas as gpd

gdf = gpd.read_file('DEPARTEMENT.shp')
#gdf['new_column'] = gdf['INSEE_DEP']
#gdf_new = gdf.dissolve(by='new_column')
#gdf.INSEE_DEP.to_list()
#gdf[gdf.INSEE_DEP.isin(['2A', '2B'])] = '20'
#gdf_new = gdf.dissolve(by='INSEE_DEP')
gdf['INSEE_DEP'] = np.where(gdf['INSEE_DEP'].isin(['2A', '2B']), '20', gdf['INSEE_DEP'])

gdf_new = gdf.dissolve(by='INSEE_DEP').reset_index()
gdf_metro_corse_merge = gdf_new[~gdf_new.INSEE_DEP.str.startswith('97')]
gdf_merged = gdf_metro_corse_merge.merge(res, how='left', left_on='INSEE_DEP', right_on='dep')
gdf_merged = gdf_merged.to_crs(2154)
gdf_merged.to_file('abeilles.gpkg', driver='GPKG', layer='depts_abeilles_2016')
gdf_merged.geometry = gdf_merged.representative_point()
gdf_merged.to_file('abeilles.gpkg', driver='GPKG', layer='depts_points_abeilles_2016')
gdf_merged

,INSEE_DEP,geometry,ID,NOM_DEP,NOM_DEP_M,INSEE_REG,CHF_DEP,reg,dep,nb_decl_1_a_10_ruches,...,nb_decl_sans_siret_50_a_199_ruches,nb_decl_sans_siret_500_a_999_ruches,nb_decl_sans_siret_plus_de_1000_ruches,nb_ruches_1_a_10_ruches,nb_ruches_11_a_49_ruches,nb_ruches_200_a_499_ruches,nb_ruches_50_a_199_ruches,nb_ruches_500_a_999_ruches,nb_ruches_plus_de_1000_ruches,tot_ruches
0,01,POINT (876703.294 6554527.150),DEP000000000000000000001,Ain,AIN,84,01053,AUVERGNE-RHONE-ALPES,01,548.0,...,1.0,0.0,NaN,2998.0,3692.0,4417.0,2906.0,5557.0,NaN,19570.0
1,02,POINT (744020.360 6928394.600),DEP000000000000000000002,Aisne,AISNE,32,02408,HAUTS-DE-FRANCE,02,244.0,...,0.0,NaN,NaN,1340.0,1477.0,1476.0,1183.0,NaN,NaN,5476.0
2,03,POINT (712558.582 6585396.000),DEP000000000000000000003,Allier,ALLIER,84,03190,AUVERGNE-RHONE-ALPES,03,266.0,...,1.0,0.0,NaN,1359.0,1380.0,953.0,1047.0,845.0,NaN,5584.0
3,04,POINT (955437.973 6346754.200),DEP000000000000000000004,Alpes-de-Haute-Provence,ALPES DE HAUTE PROVENCE,93,04070,PROVENCE-ALPES-COTE D'AZUR,04,169.0,...,6.0,0.0,0.0,982.0,2653.0,16452.0,5801.0,4538.0,1160.0,31586.0
4,05,POINT (964829.096 6400110.350),DEP000000000000000000005,Hautes-Alpes,HAUTES ALPES,93,05061,PROVENCE-ALPES-COTE D'AZUR,05,262.0,...,0.0,0.0,NaN,1722.0,3452.0,7516.0,5132.0,552.0,NaN,18374.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,91,POINT (643974.601 6825916.300),DEP000000000000000000092,Essonne,ESSONNE,11,91228,ILE-DE-FRANCE,91,342.0,...,0.0,NaN,NaN,1694.0,1122.0,529.0,1295.0,NaN,NaN,4640.0
91,92,POINT (641191.257 6860245.550),DEP000000000000000000093,Hauts-de-Seine,HAUTS DE SEINE,11,92050,ILE-DE-FRANCE,92,197.0,...,0.0,NaN,NaN,797.0,310.0,338.0,221.0,NaN,NaN,1666.0
92,93,POINT (659935.274 6867905.600),DEP000000000000000000094,Seine-Saint-Denis,SEINE SAINT DENIS,11,93008,ILE-DE-FRANCE,93,102.0,...,0.0,NaN,NaN,468.0,510.0,NaN,231.0,NaN,NaN,1209.0
93,94,POINT (659883.362 6852879.750),DEP000000000000000000095,Val-de-Marne,VAL DE MARNE,11,94028,ILE-DE-FRANCE,94,183.0,...,NaN,NaN,NaN,789.0,637.0,230.0,NaN,NaN,NaN,1656.0
